In [2]:
# default_exp modules.data

In [53]:
#export
import numpy as np
import glob
import os
import uproot as ur
import time
from multiprocessing import Process, Queue, set_start_method
import compress_pickle as pickle
from scipy.stats import circmean
import random

import pandas as pd
pd.set_option('display.max_columns', None)

# GraphDataGenerator

> API details.

In [5]:
data_dir = '/clusterfs/ml4hep/mfong/ML4Pions/MLTreeData/'
out_dir = '/clusterfs/ml4hep/mfong/ML4Pions/graph_data_tracks/'
pi0_files = np.sort(glob.glob(data_dir+'*pi0*/*.root'))[10:20]
pion_files = np.sort(glob.glob(data_dir+'*pion*/*.root'))[10:20]

In [28]:
cell_geo_df = ur.open(data_dir+"CellGeo.neighbours.root", library="pd")["CellGeo"].arrays(library="pd")

In [29]:
ur.open(data_dir+"CellGeo.neighbours.root")["CellGeo"].arrays(library="pd")

cell_geo_ID  cell_geo_sampling  cell_geo_eta  cell_geo_phi  \
entry subentry                                                               
0     0           740294656                  6     -2.559710      0.053900   
      1           740294658                  6     -2.559648      0.151909   
      2           740294660                  6     -2.559603      0.249912   
      3           740294662                  6     -2.559574      0.347912   
      4           740294664                  6     -2.559562      0.445909   
...                     ...                ...           ...           ...   
      187645     1284491536                 15      0.958372     -0.049087   
      187646     1284491824                 17      1.058902     -0.049087   
      187647     1284492080                 17      1.159304     -0.049087   
      187648     1284492592                 17      1.309847     -0.049087   
      187649     1284493104                 17      1.510363     -0.049087   

                cell_geo_rPerp  cell_geo_deta  cell_geo_dphi  cell_geo_volume  \
entry subentry                                                                  
0     0             617.735962            0.1       0.098175     1.697610e+06   
      1             617.774719            0.1       0.098175     1.697610e+06   
      2             617.803223            0.1       0.098175     1.697610e+06   
      3             617.821167            0.1       0.098175     1.697610e+06   
      4             617.828552            0.1       0.098175     1.697610e+06   
...                        ...            ...            ...              ...   
      187645       3215.000000            0.1       0.098175     1.346147e+07   
      187646       2809.000000            0.1       0.098175     1.341334e+06   
      187647       2477.000000            0.1       0.098175     1.241210e+06   
      187648       2060.000000            0.2       0.098175     7.739876e+05   
      187649       1640.000000            0.2       0.098175     4.666501e+05   

                cell_geo_sigma  cell_geo_prevInPhi  cell_geo_nextInPhi  \
entry subentry                                                           
0     0              49.457161                  63                   1   
      1              49.457161                   0                   2   
      2              49.457161                   1                   3   
      3              49.457161                   2                   4   
      4              49.457161                   3                   5   
...                        ...                 ...                 ...   
      187645         20.233513              187639              187268   
      187646         10.413343              187640              187269   
      187647         10.957963              187641              187270   
      187648         12.509910              187642              187271   
      187649         11.231500              187643              187272   

                cell_geo_prevInEta  cell_geo_nextInEta  cell_geo_prevInSamp  \
entry subentry                                                                
0     0                         -1                  64                   -1   
      1                         -1                  65                   -1   
      2                         -1                  66                   -1   
      3                         -1                  67                   -1   
      4                         -1                  68                   -1   
...                            ...                 ...                  ...   
      187645                    -1              186872                   -1   
      187646                185346              186872                   -1   
      187647                185347              186874               187648   
      187648                    -1                  -1               187649   
      187649                    -1  

In [56]:
test_pi0 = ur.open(pi0_files[0])["EventTree"].arrays(library="np")

In [57]:
test_pi0_df = pd.DataFrame(test_pi0_df)

In [59]:
test_pi0_df

,runNumber,eventNumber,lumiBlock,coreFlags,mcEventNumber,mcChannelNumber,mcEventWeight,nTruthPart,G4PreCalo_n_EM,G4PreCalo_E_EM,G4PreCalo_n_Had,G4PreCalo_E_Had,truthVertexX,truthVertexY,truthVertexZ,truthPartPdgId,truthPartStatus,truthPartBarcode,truthPartPt,truthPartE,truthPartMass,truthPartEta,truthPartPhi,nTrack,trackPt,trackP,trackMass,trackEta,trackPhi,trackNumberOfPixelHits,trackNumberOfSCTHits,trackNumberOfPixelDeadSensors,trackNumberOfSCTDeadSensors,trackNumberOfPixelSharedHits,trackNumberOfSCTSharedHits,trackNumberOfPixelHoles,trackNumberOfSCTHoles,trackNumberOfInnermostPixelLayerHits,trackNumberOfNextToInnermostPixelLayerHits,trackExpectInnermostPixelLayerHit,trackExpectNextToInnermostPixelLayerHit,trackNumberOfTRTHits,trackNumberOfTRTOutliers,trackChiSquared,trackNumberDOF,trackD0,trackZ0,trackEta_PreSamplerB,trackPhi_PreSamplerB,trackEta_PreSamplerE,trackPhi_PreSamplerE,trackEta_EMB1,trackPhi_EMB1,trackEta_EMB2,trackPhi_EMB2,trackEta_EMB3,trackPhi_EMB3,trackEta_EME1,trackPhi_EME1,trackEta_EME2,trackPhi_EME2,trackEta_EME3,trackPhi_EME3,trackEta_HEC0,trackPhi_HEC0,trackEta_HEC1,trackPhi_HEC1,trackEta_HEC2,trackPhi_HEC2,trackEta_HEC3,trackPhi_HEC3,trackEta_TileBar0,trackPhi_TileBar0,trackEta_TileBar1,trackPhi_TileBar1,trackEta_TileBar2,trackPhi_TileBar2,trackEta_TileGap1,trackPhi_TileGap1,trackEta_TileGap2,trackPhi_TileGap2,trackEta_TileGap3,trackPhi_TileGap3,trackEta_TileExt0,trackPhi_TileExt0,trackEta_TileExt1,trackPhi_TileExt1,trackEta_TileExt2,trackPhi_TileExt2,AntiKt4EMTopoJetsPt,AntiKt4EMTopoJetsEta,AntiKt4EMTopoJetsPhi,AntiKt4EMTopoJetsE,AntiKt4LCTopoJetsPt,AntiKt4LCTopoJetsEta,AntiKt4LCTopoJetsPhi,AntiKt4LCTopoJetsE,AntiKt4TruthJetsPt,AntiKt4TruthJetsEta,AntiKt4TruthJetsPhi,AntiKt4TruthJetsE,AntiKt4TruthJetsFlavor,nCluster,cluster_E,cluster_E_LCCalib,cluster_Pt,cluster_Eta,cluster_Phi,cluster_nCells,cluster_ENG_CALIB_TOT,cluster_ENG_CALIB_OUT_T,cluster_ENG_CALIB_DEAD_TOT,cluster_EM_PROBABILITY,cluster_HAD_WEIGHT,cluster_OOC_WEIGHT,cluster_DM_WEIGHT,cluster_CENTER_MAG,cluster_FIRST_ENG_DENS,cluster_CENTER_LAMBDA,cluster_ISOLATION,cluster_ENERGY_DigiHSTruth,cluster_cell_ID,cluster_cell_E,cluster_hitsTruthIndex,cluster_hitsTruthE
0,284500,140133,1,0,140133,900246,1.0,3,0,0.000000,0,0.0,-0.498615,-0.490637,-48.252880,"[111, 22, 22]","[2, 1, 1]","[10001, 10002, 10003]","[4.4773717, 3.027036, 1.4510545]","[31.195623, 21.434145, 9.761479]","[0.135, -4.2286396e-07, -2.9901e-07]","[2.6291893, 2.6455247, 2.59373]","[2.3084373, 2.2960308, 2.3343208]",0,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],2,"[31.147184, 0.8699822]","[43.42541, 2.1540976]","[4.44873, 0.24599135]","[2.6341128, 1.9357086]","[2.3160527, 0.9339787]","[17, 10]","[29.593943, 0.0]","[1.4118366, 0.0]","[0.18440042, 0.0]","[0.59844816, 4.9248203e-05]","[1.0283419, 1.0]","[1.2489712, 2.3308444]","[1.0855134, 1.0622867]","[4005.3567, 6141.9746]","[7.435823e-06, 1.094638e-07]","[217.00365, 2242.5227]","[1.0, 1.0]","[-999.0, -999.0]","((782238254, 782238252, 782238256, 782237742, ...","((16.006433, 1.4903948, 1.742844, 7.070879, 0....","((1, 2), ())","((20.38507, 9.073153), ())"
1,284500,140970,1,0,140970,900246,1.0,3,0,0.000000,0,0.0,-0.490041,-0.486448,-90.519485,"[111, 22, 22]","[2, 1, 1]","[10001, 10002, 10003]","[1.222037, 0.25063244, 0.97145665]","[1.2363178, 0.2636745, 0.9726433]","[0.135, -3.8146974e-09, 0.0]","[0.1061249, 0.32122105, 0.049423188]","[2.320521, 2.3023481, 2.3252094]",0,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],1,[0.514682],[0.93998235],[0.5146507],[-0.011033695],[2.3146243],[17],[0.49528614],[0.1669362],[0.33325565],[0.5468768],[1.0528158],[1.7344221],[1.0001693]

In [64]:
test_pi0_df.columns.values

array(['runNumber', 'eventNumber', 'lumiBlock', 'coreFlags',
       'mcEventNumber', 'mcChannelNumber', 'mcEventWeight', 'nTruthPart',
       'G4PreCalo_n_EM', 'G4PreCalo_E_EM', 'G4PreCalo_n_Had',
       'G4PreCalo_E_Had', 'truthVertexX', 'truthVertexY', 'truthVertexZ',
       'truthPartPdgId', 'truthPartStatus', 'truthPartBarcode',
       'truthPartPt', 'truthPartE', 'truthPartMass', 'truthPartEta',
       'truthPartPhi', 'nTrack', 'trackPt', 'trackP', 'trackMass',
       'trackEta', 'trackPhi', 'trackNumberOfPixelHits',
       'trackNumberOfSCTHits', 'trackNumberOfPixelDeadSensors',
       'trackNumberOfSCTDeadSensors', 'trackNumberOfPixelSharedHits',
       'trackNumberOfSCTSharedHits', 'trackNumberOfPixelHoles',
       'trackNumberOfSCTHoles', 'trackNumberOfInnermostPixelLayerHits',
       'trackNumberOfNextToInnermostPixelLayerHits',
       'trackExpectInnermostPixelLayerHit',
       'trackExpectNextToInnermostPixelLayerHit', 'trackNumberOfTRTHits',
       'trackNumberOfTRTOutlier

In [65]:
test_pi0_df.head(50)

,runNumber,eventNumber,lumiBlock,coreFlags,mcEventNumber,mcChannelNumber,mcEventWeight,nTruthPart,G4PreCalo_n_EM,G4PreCalo_E_EM,G4PreCalo_n_Had,G4PreCalo_E_Had,truthVertexX,truthVertexY,truthVertexZ,truthPartPdgId,truthPartStatus,truthPartBarcode,truthPartPt,truthPartE,truthPartMass,truthPartEta,truthPartPhi,nTrack,trackPt,trackP,trackMass,trackEta,trackPhi,trackNumberOfPixelHits,trackNumberOfSCTHits,trackNumberOfPixelDeadSensors,trackNumberOfSCTDeadSensors,trackNumberOfPixelSharedHits,trackNumberOfSCTSharedHits,trackNumberOfPixelHoles,trackNumberOfSCTHoles,trackNumberOfInnermostPixelLayerHits,trackNumberOfNextToInnermostPixelLayerHits,trackExpectInnermostPixelLayerHit,trackExpectNextToInnermostPixelLayerHit,trackNumberOfTRTHits,trackNumberOfTRTOutliers,trackChiSquared,trackNumberDOF,trackD0,trackZ0,trackEta_PreSamplerB,trackPhi_PreSamplerB,trackEta_PreSamplerE,trackPhi_PreSamplerE,trackEta_EMB1,trackPhi_EMB1,trackEta_EMB2,trackPhi_EMB2,trackEta_EMB3,trackPhi_EMB3,trackEta_EME1,trackPhi_EME1,trackEta_EME2,trackPhi_EME2,trackEta_EME3,trackPhi_EME3,trackEta_HEC0,trackPhi_HEC0,trackEta_HEC1,trackPhi_HEC1,trackEta_HEC2,trackPhi_HEC2,trackEta_HEC3,trackPhi_HEC3,trackEta_TileBar0,trackPhi_TileBar0,trackEta_TileBar1,trackPhi_TileBar1,trackEta_TileBar2,trackPhi_TileBar2,trackEta_TileGap1,trackPhi_TileGap1,trackEta_TileGap2,trackPhi_TileGap2,trackEta_TileGap3,trackPhi_TileGap3,trackEta_TileExt0,trackPhi_TileExt0,trackEta_TileExt1,trackPhi_TileExt1,trackEta_TileExt2,trackPhi_TileExt2,AntiKt4EMTopoJetsPt,AntiKt4EMTopoJetsEta,AntiKt4EMTopoJetsPhi,AntiKt4EMTopoJetsE,AntiKt4LCTopoJetsPt,AntiKt4LCTopoJetsEta,AntiKt4LCTopoJetsPhi,AntiKt4LCTopoJetsE,AntiKt4TruthJetsPt,AntiKt4TruthJetsEta,AntiKt4TruthJetsPhi,AntiKt4TruthJetsE,AntiKt4TruthJetsFlavor,nCluster,cluster_E,cluster_E_LCCalib,cluster_Pt,cluster_Eta,cluster_Phi,cluster_nCells,cluster_ENG_CALIB_TOT,cluster_ENG_CALIB_OUT_T,cluster_ENG_CALIB_DEAD_TOT,cluster_EM_PROBABILITY,cluster_HAD_WEIGHT,cluster_OOC_WEIGHT,cluster_DM_WEIGHT,cluster_CENTER_MAG,cluster_FIRST_ENG_DENS,cluster_CENTER_LAMBDA,cluster_ISOLATION,cluster_ENERGY_DigiHSTruth,cluster_cell_ID,cluster_cell_E,cluster_hitsTruthIndex,cluster_hitsTruthE
0,284500,140133,1,0,140133,900246,1.0,3,0,0.000000,0,0.0,-0.498615,-0.490637,-48.252880,"[111, 22, 22]","[2, 1, 1]","[10001, 10002, 10003]","[4.4773717, 3.027036, 1.4510545]","[31.195623, 21.434145, 9.761479]","[0.135, -4.2286396e-07, -2.9901e-07]","[2.6291893, 2.6455247, 2.59373]","[2.3084373, 2.2960308, 2.3343208]",0,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],2,"[31.147184, 0.8699822]","[43.42541, 2.1540976]","[4.44873, 0.24599135]","[2.6341128, 1.9357086]","[2.3160527, 0.9339787]","[17, 10]","[29.593943, 0.0]","[1.4118366, 0.0]","[0.18440042, 0.0]","[0.59844816, 4.9248203e-05]","[1.0283419, 1.0]","[1.2489712, 2.3308444]","[1.0855134, 1.0622867]","[4005.3567, 6141.9746]","[7.435823e-06, 1.094638e-07]","[217.00365, 2242.5227]","[1.0, 1.0]","[-999.0, -999.0]","((782238254, 782238252, 782238256, 782237742, ...","((16.006433, 1.4903948, 1.742844, 7.070879, 0....","((1, 2), ())","((20.38507, 9.073153), ())"
1,284500,140970,1,0,140970,900246,1.0,3,0,0.000000,0,0.0,-0.490041,-0.486448,-90.519485,"[111, 22, 22]","[2, 1, 1]","[10001, 10002, 10003]","[1.222037, 0.25063244, 0.97145665]","[1.2363178, 0.2636745, 0.9726433]","[0.135, -3.8146974e-09, 0.0]","[0.1061249, 0.32122105, 0.049423188]","[2.320521, 2.3023481, 2.3252094]",0,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],1,[0.514682],[0.93998235],[0.5146507],[-0.011033695],[2.3146243],[17],[0.49528614],[0.1669362],[0.33325565],[0.5468768],[1.0528158],[1.7344221],[1.0001693]

In [67]:
ur.open(pi0_files[0])["EventTree"].show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
runNumber            | int32_t                  | AsDtype('>i4')
eventNumber          | int64_t                  | AsDtype('>i8')
lumiBlock            | int32_t                  | AsDtype('>i4')
coreFlags            | uint32_t                 | AsDtype('>u4')
mcEventNumber        | int32_t                  | AsDtype('>i4')
mcChannelNumber      | int32_t                  | AsDtype('>i4')
mcEventWeight        | float                    | AsDtype('>f4')
nTruthPart           | int32_t                  | AsDtype('>i4')
G4PreCalo_n_EM       | int32_t                  | AsDtype('>i4')
G4PreCalo_E_EM       | float                    | AsDtype('>f4')
G4PreCalo_n_Had      | int32_t                  | AsDtype('>i4')
G4PreCalo_E_Had      | float                    | AsDtype('>f4')
truthVertexX         | float                    | AsDtype(

In [ ]:
# TODO
# Add nTrack	trackPt	trackP	trackMass	trackEta	trackPhi	
# as nodes

In [4]:
#hide
from nbdev.showdoc import *

In [9]:
#export
class GraphDataGenerator:
    """
    DataGenerator class for extracting and formating data from list of root files
    This data generator uses the cell_geo file to create the input graph structure
    """
    def __init__(self,
                 pi0_file_list: list,
                 pion_file_list: list,
                 cellGeo_file: str,
                 batch_size: int,
                 shuffle: bool = True,
                 num_procs = 32,
                 preprocess = False,
                 output_dir = None):
        """Initialization"""

        self.preprocess = preprocess
        self.output_dir = output_dir

        if self.preprocess and self.output_dir is not None:
            self.pi0_file_list = pi0_file_list
            self.pion_file_list = pion_file_list
            assert len(pi0_file_list) == len(pion_file_list)
            self.num_files = len(self.pi0_file_list)
        else:
            self.file_list = pi0_file_list
            self.num_files = len(self.file_list)
        
        self.cellGeo_file = cellGeo_file
        
        self.cellGeo_data = ur.open(self.cellGeo_file)['CellGeo']
        self.geoFeatureNames = self.cellGeo_data.keys()[1:9]
        self.nodeFeatureNames = ['cluster_cell_E', *self.geoFeatureNames[:-2]]
        self.edgeFeatureNames = self.cellGeo_data.keys()[9:]
        self.num_nodeFeatures = len(self.nodeFeatureNames)
        self.num_edgeFeatures = len(self.edgeFeatureNames)

        self.cellGeo_data = self.cellGeo_data.arrays(library='np')
        self.cellGeo_ID = self.cellGeo_data['cell_geo_ID'][0]
        self.sorter = np.argsort(self.cellGeo_ID)
        
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        if self.shuffle: np.random.shuffle(self.file_list)
        
        self.num_procs = np.min([num_procs, self.num_files])
        self.procs = []

        if self.preprocess and self.output_dir is not None:
            os.makedirs(self.output_dir, exist_ok=True)
            self.preprocess_data()

    def get_cluster_calib(self, event_data, event_ind, cluster_ind):
        """ Reading cluster calibration energy """ 
            
        cluster_calib_E = event_data['cluster_ENG_CALIB_TOT'][event_ind][cluster_ind]

        if cluster_calib_E <= 0:
            return None

        return np.log10(cluster_calib_E)
            
    def get_nodes(self, event_data, event_ind, cluster_ind):
        """ Reading Node features """ 

        cell_IDs = event_data['cluster_cell_ID'][event_ind][cluster_ind]
        cell_IDmap = self.sorter[np.searchsorted(self.cellGeo_ID, cell_IDs, sorter=self.sorter)]
        
        nodes = np.log10(event_data['cluster_cell_E'][event_ind][cluster_ind])
        global_node = np.log10(event_data['cluster_E'][event_ind][cluster_ind])
        
        # Scaling the cell_geo_sampling by 28
        nodes = np.append(nodes, self.cellGeo_data['cell_geo_sampling'][0][cell_IDmap]/28.)
        for f in self.nodeFeatureNames[2:4]:
            nodes = np.append(nodes, self.cellGeo_data[f][0][cell_IDmap])
        # Scaling the cell_geo_rPerp by 3000
        nodes = np.append(nodes, self.cellGeo_data['cell_geo_rPerp'][0][cell_IDmap]/3000.)
        for f in self.nodeFeatureNames[5:]:
            nodes = np.append(nodes, self.cellGeo_data[f][0][cell_IDmap])

        nodes = np.reshape(nodes, (len(self.nodeFeatureNames), -1)).T
        cluster_num_nodes = len(nodes)
        
        return nodes, np.array([global_node]), cluster_num_nodes, cell_IDmap
    
    def get_edges(self, cluster_num_nodes, cell_IDmap):
        """ 
        Reading edge features 
        Returns senders, receivers, and edges    
        """ 
        
        edge_inds = np.zeros((cluster_num_nodes, self.num_edgeFeatures))
        for i, f in enumerate(self.edgeFeatureNames):
            edge_inds[:, i] = self.cellGeo_data[f][0][cell_IDmap]
        edge_inds[np.logical_not(np.isin(edge_inds, cell_IDmap))] = np.nan
        
        senders, edge_on_inds = np.isin(edge_inds, cell_IDmap).nonzero()
        cluster_num_edges = len(senders)
        edges = np.zeros((cluster_num_edges, self.num_edgeFeatures))
        edges[np.arange(cluster_num_edges), edge_on_inds] = 1
        
        cell_IDmap_sorter = np.argsort(cell_IDmap)
        rank = np.searchsorted(cell_IDmap, edge_inds , sorter=cell_IDmap_sorter)
        receivers = cell_IDmap_sorter[rank[rank!=cluster_num_nodes]]
        
        return senders, receivers, edges

    def preprocessor(self, worker_id):
        """
        Prerocessing root file data for faster data 
        generation during multiple training epochs
        """
        file_num = worker_id
        while file_num < self.num_files:
            print(f"Processing file number {file_num}")
            file = self.pion_file_list[file_num]
            event_tree = ur.open(file)['EventTree']
            num_events = event_tree.num_entries

            event_data = event_tree.arrays(library='np')

            preprocessed_data = []

            for event_ind in range(num_events):
                num_clusters = event_data['nCluster'][event_ind]
                
                for i in range(num_clusters):
                    cluster_calib_E = self.get_cluster_calib(event_data, event_ind, i)
                    
                    if cluster_calib_E is None:
                        continue
                        
                    nodes, global_node, cluster_num_nodes, cell_IDmap = self.get_nodes(event_data, event_ind, i)
                    senders, receivers, edges = self.get_edges(cluster_num_nodes, cell_IDmap)

                    graph = {'nodes': nodes.astype(np.float32), 'globals': global_node.astype(np.float32),
                        'senders': senders.astype(np.int32), 'receivers': receivers.astype(np.int32),
                        'edges': edges.astype(np.float32)}
                    target = np.reshape([cluster_calib_E.astype(np.float32), 1], [1,2])

                    preprocessed_data.append((graph, target))

            file = self.pi0_file_list[file_num]
            event_tree = ur.open(file)['EventTree']
            num_events = event_tree.num_entries

            event_data = event_tree.arrays(library='np')

            for event_ind in range(num_events):
                num_clusters = event_data['nCluster'][event_ind]
                
                for i in range(num_clusters):
                    cluster_calib_E = self.get_cluster_calib(event_data, event_ind, i)
                    
                    if cluster_calib_E is None:
                        continue
                        
                    nodes, global_node, cluster_num_nodes, cell_IDmap = self.get_nodes(event_data, event_ind, i)
                    senders, receivers, edges = self.get_edges(cluster_num_nodes, cell_IDmap)
                    
                    graph = {'nodes': nodes.astype(np.float32), 'globals': global_node.astype(np.float32),
                        'senders': senders.astype(np.int32), 'receivers': receivers.astype(np.int32),
                        'edges': edges.astype(np.float32)}
                    target = np.reshape([cluster_calib_E.astype(np.float32), 0], [1,2])

                    preprocessed_data.append((graph, target))

            random.shuffle(preprocessed_data)

            pickle.dump(preprocessed_data, open(self.output_dir + f'data_{file_num:03d}.p', 'wb'), compression='gzip')
            
            print(f"Finished processing {file_num} files")
            file_num += self.num_procs

    def preprocess_data(self):
        print('\nPreprocessing and saving data to {}'.format(self.output_dir))
        for i in range(self.num_procs):
            p = Process(target=self.preprocessor, args=(i,), daemon=True)
            p.start()
            self.procs.append(p)
        
        for p in self.procs:
            p.join()

        self.file_list = [self.output_dir + f'data_{i:03d}.p' for i in range(self.num_files)]

    def preprocessed_worker(self, worker_id, batch_queue):
        batch_graphs = []
        batch_targets = []

        file_num = worker_id
        while file_num < self.num_files:
            file_data = pickle.load(open(self.file_list[file_num], 'rb'), compression='gzip')

            for i in range(len(file_data)):
                batch_graphs.append(file_data[i][0])
                batch_targets.append(file_data[i][1])
                    
                if len(batch_graphs) == self.batch_size:
                    batch_targets = np.reshape(np.array(batch_targets), [-1,2]).astype(np.float32)
                    
                    batch_queue.put((batch_graphs, batch_targets))
                    
                    batch_graphs = []
                    batch_targets = []

            file_num += self.num_procs
                    
        if len(batch_graphs) > 0:
            batch_targets = np.reshape(np.array(batch_targets), [-1,2]).astype(np.float32)
            
            batch_queue.put((batch_graphs, batch_targets))

    def worker(self, worker_id, batch_queue):
        if self.preprocess:
            self.preprocessed_worker(worker_id, batch_queue)
        else:
            raise Exception('Preprocessing is required for combined classification/regression models.')
        
    def check_procs(self):
        for p in self.procs:
            if p.is_alive(): return True
        
        return False

    def kill_procs(self):
        for p in self.procs:
            p.kill()

        self.procs = []
    
    def generator(self):
        """
        Generator that returns processed batches during training
        """
        batch_queue = Queue(2 * self.num_procs)
            
        for i in range(self.num_procs):
            p = Process(target=self.worker, args=(i, batch_queue), daemon=True)
            p.start()
            self.procs.append(p)
        
        while self.check_procs() or not batch_queue.empty():
            try:
                batch = batch_queue.get(True, 0.0001)
            except:
                continue
            
            yield batch
        
        for p in self.procs:
            p.join()

In [6]:
data_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/'
out_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/'
pi0_files = np.sort(glob.glob(data_dir+'*pi0*/*.root'))[10:20]
pion_files = np.sort(glob.glob(data_dir+'*pion*/*.root'))[10:20]

In [7]:
data_gen = GraphDataGenerator(pion_file_list=pion_files, 
                              pi0_file_list=pi0_files,
                              cellGeo_file=data_dir+'cell_geo.root',
                              batch_size=32,
                              shuffle=False,
                              num_procs=32,
                              preprocess=True,
                              output_dir=out_dir)

# gen = data_gen.generator()


Preprocessing and saving data to /clusterfs/ml4hep/mpettee/ml4pions/data/
Proceesing file number 0
Proceesing file number 1Proceesing file number 2

Proceesing file number 3
Proceesing file number 4
Proceesing file number 5
Proceesing file number 6
Proceesing file number 7
Proceesing file number 8
Proceesing file number 9
Finished processing 3 files
Finished processing 9 files
Finished processing 6 files
Finished processing 8 files
Finished processing 2 files
Finished processing 4 files
Finished processing 7 files
Finished processing 1 files
Finished processing 5 files
Finished processing 0 files


In [8]:
out_dir = '/clusterfs/ml4hep/mpettee/ml4pions/data/'
pi0_files = np.sort(glob.glob(data_dir+'*pi0*/*.root'))[20:30]
pion_files = np.sort(glob.glob(data_dir+'*pion*/*.root'))[20:30]
data_gen_test = GraphDataGenerator(pion_file_list=pion_files, 
                                   pi0_file_list=pi0_files,
                                   cellGeo_file=data_dir+'cell_geo.root',
                                   batch_size=32,
                                   shuffle=False,
                                   num_procs=32,
                                   preprocess=True,
                                   output_dir=out_dir)


Preprocessing and saving data to /clusterfs/ml4hep/mpettee/ml4pions/data/
Proceesing file number 0
Proceesing file number 1
Proceesing file number 2
Proceesing file number 3Proceesing file number 4

Proceesing file number 5
Proceesing file number 6
Proceesing file number 7
Proceesing file number 8
Proceesing file number 9
Finished processing 6 files
Finished processing 8 files
Finished processing 1 files
Finished processing 0 files
Finished processing 5 files
Finished processing 3 files
Finished processing 4 files
Finished processing 9 files
Finished processing 2 files
Finished processing 7 files
